In [1]:
import os
import tensorflow as tf
import numpy as np
import DEN
from tensorflow.examples.tutorials.mnist import input_data
import sys

np.random.seed(1004)

flags = tf.app.flags
flags.DEFINE_integer("max_iter", 1000, "Epoch to train")
flags.DEFINE_float("lr", 0.001, "Learing rate(init) for train")
flags.DEFINE_integer("batch_size", 64, "The size of batch for 1 iteration")
flags.DEFINE_string("checkpoint_dir", "checkpoints", "Directory path to save the checkpoints")
flags.DEFINE_integer("dims0", 98, "Dimensions about input layer")
flags.DEFINE_integer("dims1", 60, "Dimensions about 1st layer")
flags.DEFINE_integer("dims2", 30, "Dimensions about 2nd layer")
flags.DEFINE_integer("dims3", 17, "Dimensions about output layer")
flags.DEFINE_integer("n_classes", 17, 'The number of classes at each task')
flags.DEFINE_float("l1_lambda", 0.00001, "Sparsity for L1")
flags.DEFINE_float("l2_lambda", 0.0001, "L2 lambda")
flags.DEFINE_float("gl_lambda", 0.001, "Group Lasso lambda")
flags.DEFINE_float("regular_lambda", 0.05, "regularization lambda")
flags.DEFINE_integer("ex_k", 200, "The number of units increased in the expansion processing")
flags.DEFINE_float('loss_thr', 0.01, "Threshold of dynamic expansion")
flags.DEFINE_float('spl_thr', 0.05, "Threshold of split and duplication")
FLAGS = flags.FLAGS

In [2]:
import pickle
awid_tasks = pickle.load(open('awid_tasks.pkl', "rb"))
n_tasks = len(awid_tasks)

In [3]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

import pdb

model = DEN.DEN(FLAGS)
params = dict()
avg_perf = []

for m in range(0,n_tasks): #(FLAGS.n_classes):
    tasked_input = awid_tasks[m][0]
    tasked_output = awid_tasks[m][1]
    print (tasked_input.shape,tasked_output.shape)
    data = (awid_tasks[m][0],awid_tasks[m][1],awid_tasks[m][2],awid_tasks[m][3],awid_tasks[m][4],awid_tasks[m][5])
    model.sess = tf.Session()
    print("\n\n\tTASK %d TRAINING\n"%(m+1))

    model.T = model.T+1
    model.task_indices.append(m+1)
    model.load_params(params, time = 1)
    perf, sparsity, expansion = model.add_task(m+1, data)
    
    params = model.get_params()
    #pdb.set_trace()
    #analyze_params(params)
    model.destroy_graph()
    model.sess.close()

    model.sess= tf.Session()
    print('\n OVERALL EVALUATION')
    model.load_params(params)
    temp_perfs = []
    for j in range(m+1):
        temp_perf = model.predict_perform(j+1, tasked_input, tasked_output)
        temp_perfs.append(temp_perf)
    avg_perf.append( sum(temp_perfs) / float(m+1) )
    print("   [*] avg_perf: %.4f"%avg_perf[m])
    model.destroy_graph()
    model.sess.close()



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(14083, 98) (14083, 17)


	TASK 1 TRAINING



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



 [*] iter: 100, val loss: 0.0719, val perf: 0.9560
 [*] iter: 200, val loss: 0.0321, val perf: 0.9693
 [*] iter: 300, val loss: 0.0242, val perf: 0.9832
 [*] iter: 400, val loss: 0.0197, val perf: 0.9903
 [*] iter: 500, val loss: 0.0147, val perf: 0.9929
 [*] iter: 600, val loss: 0.0114, val perf: 0.9947
 [*] iter: 700, val loss: 0.0098, val perf: 0.9952
 [*] iter: 800, val loss: 0.0090, val perf: 0.9956
 [*] iter: 900, val loss: 0.0086, val perf: 0.9960
 [*] iter: 1000, val loss: 0.0081, val perf: 0.9961
   [*] Total expansions: [0, 0]
 [*] hidden 1, shape : [98, 60]
 [*] hidden 2, shape : [60, 30]
 [*] Task: 1, nn_test_loss: 0.0081, test_perf: 0.9961, sparsity(avg): 0.6533

 OVERALL EVALUATION
 [*] hidden 1, shape : [98, 60]


 [*] Evaluation, Task:2, test_acc: 0.5140
 [*] hidden 1, shape : [98, 121]
 [*] hidden 2, shape : [121, 229]
 [*] Evaluation, Task:3, test_acc: 0.5391
 [*] hidden 1, shape : [98, 136]
 [*] hidden 2, shape : [136, 314]
 [*] Evaluation, Task:4, test_acc: 0.5350
 [*] hidden 1, shape : [98, 153]
 [*] hidden 2, shape : [153, 386]
 [*] Evaluation, Task:5, test_acc: 1.0000
   [*] avg_perf: 0.6437
